# Traditional PCA

In [1]:
load hald

[coeff,score,latent,~,~,mu] = pca(ingredients);

[dataLen,d] = size(ingredients); % D for number of dimensions
reconstructedData = score*coeff'+repmat(mu,dataLen,1);

ingredients

reconstructedData

% pca is just taking the eigenvalue decomposition of the covariance matrix
myMu = mean(ingredients,1);
tempData = bsxfun(@minus,ingredients,myMu);
[V,D] = eig(cov(tempData)); % won't necessarily order the eigenvalues like
          % matlab's pca function does
% V is now the exact same thing as coeff above (just not in the right order)
myScore = tempData/V';
myReconstructedData = myScore*V'+repmat(myMu,dataLen,1);

myReconstructedData

ingredients =

     7    26     6    60
     1    29    15    52
    11    56     8    20
    11    31     8    47
     7    52     6    33
    11    55     9    22
     3    71    17     6
     1    31    22    44
     2    54    18    22
    21    47     4    26
     1    40    23    34
    11    66     9    12
    10    68     8    12


reconstructedData =

    7.0000   26.0000    6.0000   60.0000
    1.0000   29.0000   15.0000   52.0000
   11.0000   56.0000    8.0000   20.0000
   11.0000   31.0000    8.0000   47.0000
    7.0000   52.0000    6.0000   33.0000
   11.0000   55.0000    9.0000   22.0000
    3.0000   71.0000   17.0000    6.0000
    1.0000   31.0000   22.0000   44.0000
    2.0000   54.0000   18.0000   22.0000
   21.0000   47.0000    4.0000   26.0000
    1.0000   40.0000   23.0000   34.0000
   11.0000   66.0000    9.0000   12.0000
   10.0000   68.0000    8.0000   12.0000


myReconstructedData =

    7.0000   26.0000    6.0000   60.0000
    1.0000   29.0000   15.0000   52.00

Maintaining the full dimensionality in "score", you can easily recover the original data. 

# Dimensionality Reduction

In [17]:
reduceCoeff = coeff(:,1:3);
reduceScore = score(:,1:3);
dimReduceReconstructedData = reduceScore*reduceCoeff'+repmat(mu,dataLen,1);

ingredients

dimReduceReconstructedData

avError = mean(mean(abs(ingredients-dimReduceReconstructedData)));
fprintf('Average absolute error in reconstruction of original data: %3.2f\n',avError);

avRelError = mean(mean(abs(ingredients-dimReduceReconstructedData)./ingredients));
fprintf('Average relative error in reconstruction of original data: %3.3f\n',avRelError);

ingredients =

     7    26     6    60
     1    29    15    52
    11    56     8    20
    11    31     8    47
     7    52     6    33
    11    55     9    22
     3    71    17     6
     1    31    22    44
     2    54    18    22
    21    47     4    26
     1    40    23    34
    11    66     9    12
    10    68     8    12


dimReduceReconstructedData =

    6.7992   25.8043    5.7955   59.8079
    1.2004   29.1953   15.2041   52.1918
   11.5700   56.5555    8.5806   20.5455
   11.1916   31.1867    8.1952   47.1834
    6.9279   51.9298    5.9266   32.9310
   11.0683   55.0666    9.0696   22.0654
    2.9586   70.9597   16.9578    5.9604
    0.8358   30.8400   21.8328   43.8429
    2.2752   54.2682   18.2803   22.2634
   20.8276   46.8320    3.8244   25.8351
    0.7797   39.7854   22.7756   33.7892
   10.7738   65.7796    8.7696   11.7836
    9.7917   67.7970    7.7878   11.8006

Average absolute error in reconstruction of original data: 0.20
Average relative error in reco

The recovered data is almost identical to the original. Thus, the reduceScore matrix is a lower-dimensional representation of the original data.

# Probabilistic PCA

In [2]:
[dataLen,d] = size(ingredients); % D for number of dimensions

[V,D] = eig(cov(ingredients));
sigmasquare = 0;%(1/(d-q))*sum(dropped eigenvalues) and q is number of dropped eigenvalues
W = V*sqrtm(D-sigmasquare.*eye(d));

fprintf('W provides a full representation of the covariance matrix C\n');
cov(ingredients)
size(W)
W*W'+sigmasquare.*eye(d)

fprintf('Drop the smallest eigenvalue to yield a lower-rank representation\n');
% drop the first and smallest eigenvalue
eigenvalues = diag(D);
sigmasquare = D(1,1);q = 3;
W = V(:,2:end)*sqrtm(D(2:end,2:end)-sigmasquare.*eye(q));
size(W)
W*W'+sigmasquare.*eye(d)

fprintf('Drop the first two smallest eigenvalues\n');
% drop bottom two eigenvalues
q = 2;sigmasquare =(1/(d-q))*sum(eigenvalues(1:2));
W = V(:,3:end)*sqrtm(D(3:end,3:end)-sigmasquare.*eye(q));
size(W)
W*W'+sigmasquare.*eye(d)

W provides a full representation of the covariance matrix C

ans =

   34.6026   20.9231  -31.0513  -24.1667
   20.9231  242.1410  -13.8782 -253.4167
  -31.0513  -13.8782   41.0256    3.1667
  -24.1667 -253.4167    3.1667  280.1667


ans =

     4     4


ans =

   34.6026   20.9231  -31.0513  -24.1667
   20.9231  242.1410  -13.8782 -253.4167
  -31.0513  -13.8782   41.0256    3.1667
  -24.1667 -253.4167    3.1667  280.1667

Drop the smallest eigenvalue to yield a lower-rank representation

ans =

     4     3


ans =

   34.6026   20.9231  -31.0513  -24.1667
   20.9231  242.1410  -13.8782 -253.4167
  -31.0513  -13.8782   41.0256    3.1667
  -24.1667 -253.4167    3.1667  280.1667

Drop the first two smallest eigenvalues

ans =

     4     2


ans =

   34.2033   24.3198  -30.8564  -21.0581
   24.3198  241.8211  -10.9953 -253.5131
  -30.8564  -10.9953   41.6520    5.8362
  -21.0581 -253.5131    5.8362  280.2595


Dropping out the first and smallest eigenvalue has no effect on the reconstruction 
of the covariance matrix, despite the fact that the matrix W now has one fewer 
column. If we drop the first two smallest eigenvalues, then the reconstruction
of the covariance matrix causes fairly significant alterations. Thus, probabilistic
PCA (PPCA) appears to provide a means by which we can create lower-dimensional
representations of the covariance matrix of a dataset. In addition, the original
data X and be recovered via the linear transformation X = WZ+sigmasquareI , or 
X = WZ+mu+sigmasquareI (I is the identity matrix and Z is an unobserved latent
variable). 

# PPCA (cont.)

In [20]:
[V,D] = eig(cov(ingredients));
sigmasquare = 0;%(1/(d-q))*sum(dropped eigenvalues) and q is number of dropped eigenvalues
W = V*sqrtm(D-sigmasquare.*eye(d));

mu = mean(ingredients,1);

% data is now described by the following multivariate normal distribution

% N(mu,W*W'+sigmasquare*eye(d));

% the model is data = W*latent + mu + noise
%   so, data-mu-noise = W*latent
%   and inv(W)*(data-mu-noise) = latent

latentVars = inv(W)*(ingredients-repmat(mu,dataLen,1))';

reconstructedData = W*latentVars+repmat(mu,dataLen,1)';
fprintf('Reconstructed Data (Full Transformation Matrix):\n');
display(reconstructedData');

eigenvalues = diag(D);
sigmasquare = D(1,1);q = 3;
W = V(:,2:end)*sqrtm(D(2:end,2:end)-sigmasquare.*eye(q));

latentVars = pinv(W)*(ingredients-repmat(mu,dataLen,1))';

reconstructedData = W*latentVars+repmat(mu,dataLen,1)';
fprintf('Reconstructed Data (Reduced Transformation Matrix):\n');
display(reconstructedData');

avError = mean(mean(abs(ingredients-reconstructedData')));
fprintf('Average absolute error in reconstruction of original data: %3.2f\n',avError);

avRelError = mean(mean(abs(ingredients-reconstructedData')./ingredients));
fprintf('Average relative error in reconstruction of original data: %3.3f\n',avRelError);

Reconstructed Data (Full Transformation Matrix):

ans =

    7.0000   26.0000    6.0000   60.0000
    1.0000   29.0000   15.0000   52.0000
   11.0000   56.0000    8.0000   20.0000
   11.0000   31.0000    8.0000   47.0000
    7.0000   52.0000    6.0000   33.0000
   11.0000   55.0000    9.0000   22.0000
    3.0000   71.0000   17.0000    6.0000
    1.0000   31.0000   22.0000   44.0000
    2.0000   54.0000   18.0000   22.0000
   21.0000   47.0000    4.0000   26.0000
    1.0000   40.0000   23.0000   34.0000
   11.0000   66.0000    9.0000   12.0000
   10.0000   68.0000    8.0000   12.0000

Reconstructed Data (Reduced Transformation Matrix):

ans =

    6.7992   25.8043    5.7955   59.8079
    1.2004   29.1953   15.2041   52.1918
   11.5700   56.5555    8.5806   20.5455
   11.1916   31.1867    8.1952   47.1834
    6.9279   51.9298    5.9266   32.9310
   11.0683   55.0666    9.0696   22.0654
    2.9586   70.9597   16.9578    5.9604
    0.8358   30.8400   21.8328   43.8429
    2.2752   54.2682 

Note that this reconstruction is identical to the reconstruction above using traditional
PCA. The errors are also identical. So, PPCA can be used to create a lower-dimensional
representation of a dataset (as latent variables), from which the original dataset
can be almost exactly reconstructed. You could switch to the latent variable space
for analysis, for example.

# Bayesian PCA